## Check List 1 - Robots
* **1.1. indy**  
  - joint_move_make_sure, move_joint_s_curve, move_joint_wp, grasp  
* **1.2. panda**  
  - joint_move_make_sure,  move_joint_s_curve,  move_joint_wp,  grasp  
* **1.3. combined_robot**  
  - joint_move_make_sure, move_joint_wp, grasp  
  
#### [NOTE] Indy, Panda 로봇 IP는 현재 로봇 사용하는 담당자에게 문의해서 진행할 것.

## set running directory to project source

In [1]:
import os
import sys
import numpy as np
import time
sys.path.append(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

## 1.1. indy
**Turn indy on, run controller program from rnb-control project**  
1. Turn ControlBox's power on.
2. Connect to ControlBox's ssh server on terminal - ip used below is just an example
```bash
ssh user@192.168.21.6
```
3. Stop the default program
```bash
sudo killall -9 UDEVMonitor && sudo killall -9 TaskMan
```
4. Start the rnb-control version control program.
```bash
cd ~/release/IndySDK_2.3.0.1/ && sudo ./TaskManager -j indyDeploy.json
```  
5. Connect Conty and reset the robot in Conty (There is jitter a issue without conty connection)
6. Select NRIC_PD controller in ControlHub WebUI ```192.168.21.6:9990```
  * make sure gain tuning is properly done
  
  
#### WARNING: Clear the space around the robot! The robot will move.

In [2]:
from pkg.controller.trajectory_client.indy_trajectory_client import IndyTrajectoryClient
INDY_IP = "192.168.21.6"
INDY_HOME = [0, 0, -np.pi / 2, 0, -np.pi / 2, 0]

##### create IndyTrajectoryClient

In [3]:
indy = IndyTrajectoryClient(server_ip=INDY_IP)

##### joint_move_make_sure()
  * This command will move indy to given position, using functions of IndyDCPClient.

In [4]:
indy.joint_move_make_sure(INDY_HOME)

Connect: Server IP (192.168.21.6)
Connect: Server IP (192.168.21.6)


##### move_joint_s_curve(qtar, N_div=100)
  * This command will move indy to given position, with a double S velocity profile. 
  * N_div is the number of steps to divide motion

In [5]:
indy.move_joint_s_curve(np.add(INDY_HOME, [0.2]*6), N_div=100)

Connect: Server IP (192.168.21.6)
Connect: Server IP (192.168.21.6)


##### move_joint_wp()
  * This command will move indy along given waypoint positions.
  * vel_lims & acc_lims are velocity and acceleration limits for the joints. They can be both scalar and vector, in radians.
  * vel_lims & acc_lims are approaximately applied. Check the source code for detail

In [6]:
indy.move_joint_wp(
    [np.add(INDY_HOME, [0]*6), 
     np.add(INDY_HOME, [0.4]*6), 
     np.add(INDY_HOME, [-0.4]*6), 
     INDY_HOME
    ], 
    vel_lims=np.array([0.5]*6), acc_lims=np.array([1.0]*6))

Connect: Server IP (192.168.21.6)


(array([[ 2.00000000e-01,  2.00100000e-01, -1.37120000e+00,
          2.00000000e-01, -1.37120000e+00,  1.99200000e-01],
        [ 1.99734145e-01,  1.99834042e-01, -1.37146544e+00,
          1.99734145e-01, -1.37146544e+00,  1.98934972e-01],
        [ 1.98947661e-01,  1.99047251e-01, -1.37225069e+00,
          1.98947661e-01, -1.37225069e+00,  1.98150938e-01],
        ...,
        [-1.31655115e-03, -1.31655142e-03, -1.57211288e+00,
         -1.31655115e-03, -1.57211288e+00, -1.31654896e-03],
        [-3.31584246e-04, -3.31584291e-04, -1.57112791e+00,
         -3.31584246e-04, -1.57112791e+00, -3.31583881e-04],
        [ 0.00000000e+00,  0.00000000e+00, -1.57079633e+00,
          0.00000000e+00, -1.57079633e+00,  0.00000000e+00]]), 4.08)

##### grasp()
  * This command will move indy gripper, if one is attached.

In [8]:
indy.grasp(True)
time.sleep(0.5)
indy.grasp(False)
time.sleep(0.5)

Connect: Server IP (192.168.21.6)
Connect: Server IP (192.168.21.6)


## 1.2. panda
**Turn panda and host computer on, run controller program from rnb-control project**  
1. Turn panda robot and host computer power on
2. When panda is prepared, open Frank Emika Desk on a web browser (https://192.168.21.13/desk/)
3. On right side, click "unlock" icon to unlock joints.
4. Connect to host computer's ssh server 
```
ssh panda@192.168.21.14
```
5. Enable activation switch - lamp light should change to blue
6. Run panda control program in ssh
```
roslaunch panda_control joint_control_rnb.launch robot_ip:=192.168.21.13 load_gripper:=false
```
7. Select NRIC_PD controller in ControlHub WebUI ```192.168.21.14:9990```
  * make sure gain tuning is properly done
8. When the robot stops due to collision or acceleration limit, repeat 5~6 to re-start the control program  

***WARNING: Clear the space around the robot! The robot will move.***

In [2]:
from pkg.controller.trajectory_client.panda_trajectory_client import PandaTrajectoryClient
PANDA_HOST_IP = "192.168.21.14" ## The host computer is the computer that runs control program
PANDA_ROBOT_IP = "192.168.21.13" ## The robot has it's own IP
PANDA_HOME = [0, -np.pi / 8, 0, -np.pi / 2, 0, np.pi / 2, np.pi / 2]

### start roscore from other terminal before calling rospy.init_node
* this is automatically done when using SceneBuliler-GeometryScene

In [11]:
import rospy
rospy.init_node("panda_test") # ros node is initialized because panda gripper uses ros api. 
# If this cell is stuck with "Unable to register with master node" warning, run roscore on a new terminal

##### create PandaTrajectoryClient

In [4]:
panda = PandaTrajectoryClient(server_ip=PANDA_HOST_IP, robot_ip=PANDA_ROBOT_IP)

rospy.init_node() has already been called with different arguments: ('panda_test', ['/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py', '-f', '/home/rnb/.local/share/jupyter/runtime/kernel-7af4e8f9-f802-4cde-9f49-bc2292833ac7.json'], False, None, False, False)


##### joint_move_make_sure()
  * This command will move panda to given position. panda uses move_joint_s_curve

In [7]:
panda.joint_move_make_sure(PANDA_HOME)

##### move_joint_s_curve(qtar, N_div=100)
  * This command will move panda to given position, with a double S velocity profile. 
  * N_div is the number of steps to divide motion

In [8]:
panda.move_joint_s_curve(np.add(PANDA_HOME, [0.2]*7), N_div=100)
panda.joint_move_make_sure(PANDA_HOME)

##### move_joint_wp()
  * This command will move panda along given waypoint positions.
  * vel_lims & acc_lims are velocity and acceleration limits for the joints. They can be both scalar and vector, in radians.
  * vel_lims & acc_lims are approaximately applied. Check the source code for detail

In [10]:
panda.move_joint_wp(
    [np.add(PANDA_HOME, [0]*7), 
     np.add(PANDA_HOME, [0.3]*7), 
     np.add(PANDA_HOME, [-0.3]*7), 
     np.add(PANDA_HOME, [0]*7)], 
    vel_lims=np.array([0.5]*7), acc_lims=np.array([1]*7))

(array([[ 0.00000000e+00, -3.92699082e-01,  0.00000000e+00,
         -1.57079633e+00,  0.00000000e+00,  1.57079633e+00,
          1.57079633e+00],
        [ 3.19528403e-04, -3.92379553e-01,  3.19528403e-04,
         -1.57047680e+00,  3.19528403e-04,  1.57111586e+00,
          1.57111586e+00],
        [ 1.27112103e-03, -3.91427961e-01,  1.27112103e-03,
         -1.56952521e+00,  1.27112103e-03,  1.57206745e+00,
          1.57206745e+00],
        [ 2.84360973e-03, -3.89855472e-01,  2.84360973e-03,
         -1.56795272e+00,  2.84360973e-03,  1.57363994e+00,
          1.57363994e+00],
        [ 5.02488132e-03, -3.87674200e-01,  5.02488132e-03,
         -1.56577145e+00,  5.02488132e-03,  1.57582121e+00,
          1.57582121e+00],
        [ 7.80182945e-03, -3.84897252e-01,  7.80182945e-03,
         -1.56299450e+00,  7.80182945e-03,  1.57859816e+00,
          1.57859816e+00],
        [ 1.11603079e-02, -3.81538774e-01,  1.11603079e-02,
         -1.55963602e+00,  1.11603079e-02,  1.58195663e+00

##### grasp()
  * This command will move panda gripper.

In [12]:
panda.grasp(True)
time.sleep(0.5)
panda.grasp(False)
time.sleep(0.5)

## 1.3. CombinedRobot
**Turn indy and panda on**  
***WARNING: Clear the space around the robots! The robots will move.***

In [13]:
from pkg.controller.combined_robot import CombinedRobot
from pkg.controller.robot_config import RobotConfig, RobotType
INDY_IP = "192.168.21.6"
PANDA_HOST_IP = "192.168.21.14" ## The host computer is the computer that runs control program
PANDA_ROBOT_IP = "192.168.21.13" ## The robot has it's own IP

In [14]:
import rospy
rospy.init_node("panda_test") # ros node is initialized because panda gripper uses ros api. 

##### create CombinedRobot instance

In [15]:
combined_robot = CombinedRobot(
    robots_on_scene=[
        RobotConfig(0, RobotType.indy7, ((0,-0.3,0), (0,0,0)),
                    INDY_IP),
        RobotConfig(1, RobotType.panda, ((0,0.3,0), (0,0,0)),
                    "{}/{}".format(PANDA_HOST_IP, PANDA_ROBOT_IP))], 
    connection_list=[False, False])

# CombinedRobot has combined home pose.
# Default home pose for each robot is defined in pkg.controller.robot_config.RobotSpecs
COMBINED_HOME = combined_robot.home_pose 
COMBINED_JOINT_NUM = combined_robot.joint_num

connection command:
indy0: False
panda1: False
rospy.init_node() has already been called with different arguments: ('panda_test', ['/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py', '-f', '/home/rnb/.local/share/jupyter/runtime/kernel-7af4e8f9-f802-4cde-9f49-bc2292833ac7.json'], False, None, False, False)


##### reset_connection(bool, bool, ...)
* reset connection status as given as arguments

In [16]:
combined_robot.reset_connection(True, True)

connection command:
indy0: True
panda1: True
rospy.init_node() has already been called with different arguments: ('panda_test', ['/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py', '-f', '/home/rnb/.local/share/jupyter/runtime/kernel-7af4e8f9-f802-4cde-9f49-bc2292833ac7.json'], False, None, False, False)


##### joint_move_make_sure()
  * This command will move both robots to given position, one-by-one.

In [29]:
combined_robot.joint_move_make_sure(COMBINED_HOME)

Connect: Server IP (192.168.21.6)
Connect: Server IP (192.168.21.6)


##### move_joint_wp()
  * This command will move the robots along given waypoint positions, one-by-one.
  * vel_scale & acc_scale are ratio to pre-defined velocity and acceleration limits in scaler. The pre-defined values are in pkg.controller.robot_config.RobotSpecs

In [30]:
combined_robot.move_joint_wp(
    [np.add(COMBINED_HOME, [0]*COMBINED_JOINT_NUM), 
     np.add(COMBINED_HOME, [0.2]*COMBINED_JOINT_NUM), 
     np.add(COMBINED_HOME, [-0.2]*COMBINED_JOINT_NUM), 
     np.add(COMBINED_HOME, [0]*COMBINED_JOINT_NUM)], 
    vel_scale=0.2, acc_scale=0.5, error_stop=30)

Connect: Server IP (192.168.21.6)


3.8600000000000003

##### grasp(robot1=bool, robot2=bool, ...)
  * This command will move each robot gripper.

In [34]:
combined_robot.grasp(indy0=True, panda1=True)
time.sleep(0.5)
combined_robot.grasp(indy0=False, panda1=False)
time.sleep(0.5)

Connect: Server IP (192.168.21.6)
Connect: Server IP (192.168.21.6)
